In [21]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sqlite3
import numpy as np
from numpy import random
from time import time

In [22]:
#load data (make sure you have downloaded database.sqlite)
with sqlite3.connect('database.sqlite') as con:
    Player_Attributes = pd.read_sql_query("SELECT * from Player_Attributes", con)
Player_Attributes.shape

(183978, 42)

In [23]:
#select relevant fields
Player_Attributes.dropna(inplace=True)
Player_Attributes.drop(['id', 'player_fifa_api_id', 'player_api_id', 'date'], axis = 1, inplace = True)
overall_rating = Player_Attributes['overall_rating']
features = Player_Attributes.drop('overall_rating', axis = 1)
features.head()

,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,71,right,medium,medium,49,44,71,61,44,51,...,54,48,65,69,69,6,11,10,8,8
1,71,right,medium,medium,49,44,71,61,44,51,...,54,48,65,69,69,6,11,10,8,8
2,66,right,medium,medium,49,44,71,61,44,51,...,54,48,65,66,69,6,11,10,8,8
3,65,right,medium,medium,48,43,70,60,43,50,...,53,47,62,63,66,5,10,9,7,7
4,65,right,medium,medium,48,43,70,60,43,50,...,53,47,62,63,66,5,10,9,7,7


1. Use encoding to convert catogarial values to numerical data.

2. Do feature scaling on GK traits.

3. Use PCA to short the features.

4. Use GridSearch and Boosting to tune the model

5. Use r2 to comapre accuracy.

In [24]:
# Use encoding to convert catogarial values to numerical data using LabelEncoder()
from sklearn import preprocessing

le_sex = preprocessing.LabelEncoder()

#to convert into numbers

features.preferred_foot = le_sex.fit_transform(features.preferred_foot)
features.attacking_work_rate = le_sex.fit_transform(features.attacking_work_rate)
features.defensive_work_rate = le_sex.fit_transform(features.defensive_work_rate)
features.head()

# to convert back
# train.Sex = le_sex.inverse_transform(train.Sex)

,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,71,1,4,14,49,44,71,61,44,51,...,54,48,65,69,69,6,11,10,8,8
1,71,1,4,14,49,44,71,61,44,51,...,54,48,65,69,69,6,11,10,8,8
2,66,1,4,14,49,44,71,61,44,51,...,54,48,65,66,69,6,11,10,8,8
3,65,1,4,14,48,43,70,60,43,50,...,53,47,62,63,66,5,10,9,7,7
4,65,1,4,14,48,43,70,60,43,50,...,53,47,62,63,66,5,10,9,7,7


In [25]:
list(features.columns)

['potential',
 'preferred_foot',
 'attacking_work_rate',
 'defensive_work_rate',
 'crossing',
 'finishing',
 'heading_accuracy',
 'short_passing',
 'volleys',
 'dribbling',
 'curve',
 'free_kick_accuracy',
 'long_passing',
 'ball_control',
 'acceleration',
 'sprint_speed',
 'agility',
 'reactions',
 'balance',
 'shot_power',
 'jumping',
 'stamina',
 'strength',
 'long_shots',
 'aggression',
 'interceptions',
 'positioning',
 'vision',
 'penalties',
 'marking',
 'standing_tackle',
 'sliding_tackle',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_positioning',
 'gk_reflexes']

In [16]:
# with sqlite3.connect('database.sqlite') as con:
#     Player_Attributes = pd.read_sql_query("SELECT curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning from Player_Attributes", con)
# Player_Attributes.head()

,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning
0,45,39,64,49,60,64,59,47,65,55,58,54,76,35,71,70,45
1,45,39,64,49,60,64,59,47,65,55,58,54,76,35,71,70,45
2,45,39,64,49,60,64,59,47,65,55,58,54,76,35,63,41,45
3,44,38,63,48,60,64,59,46,65,54,58,54,76,34,62,40,44
4,44,38,63,48,60,64,59,46,65,54,58,54,76,34,62,40,44


In [26]:
# Feature scaling using MinMaxScaler
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
scaled_features = min_max_scaler.fit_transform(features)
scaled_features

array([[ 0.55172414,  1.        ,  0.57142857, ...,  0.09375   ,
         0.07368421,  0.07368421],
       [ 0.55172414,  1.        ,  0.57142857, ...,  0.09375   ,
         0.07368421,  0.07368421],
       [ 0.46551724,  1.        ,  0.57142857, ...,  0.09375   ,
         0.07368421,  0.07368421],
       ..., 
       [ 0.70689655,  1.        ,  0.57142857, ...,  0.75      ,
         0.2       ,  0.2       ],
       [ 0.72413793,  1.        ,  0.57142857, ...,  0.75      ,
         0.2       ,  0.2       ],
       [ 0.72413793,  1.        ,  0.57142857, ...,  0.80208333,
         0.06315789,  0.14736842]])

In [27]:
# PCA to reduce feature reduction and to improve model speed
from sklearn.decomposition import PCA
pca = PCA(n_components = 6)
pca_features = pca.fit_transform(scaled_features)
print pca.components_

[[  5.18917853e-02  -9.27765823e-02  -7.48250733e-02  -8.90595357e-03
    2.23576017e-01   2.01219313e-01   1.51120697e-01   1.93358417e-01
    2.14233750e-01   2.33263339e-01   2.33732289e-01   2.04010764e-01
    1.56564066e-01   2.21291613e-01   1.37418573e-01   1.34816262e-01
    1.34215544e-01   7.01119893e-02   1.21731023e-01   1.99229151e-01
    2.13279020e-02   1.51920920e-01   1.51783441e-02   2.29322576e-01
    1.21120402e-01   1.12343502e-01   2.24297632e-01   1.74823193e-01
    1.67140752e-01   1.02110459e-01   1.23124574e-01   1.15111506e-01
   -2.11397003e-01  -2.00154350e-01  -1.67592101e-01  -1.97108068e-01
   -2.11661286e-01]
 [  2.30371742e-02   2.10714942e-01  -4.66679956e-02   1.15594890e-02
    2.04543071e-02   2.34308732e-01  -1.10573669e-01  -1.88587839e-02
    1.91219847e-01   1.18296941e-01   1.08682310e-01   7.86790575e-02
   -6.73709990e-02   4.21657261e-02   8.69374142e-02   6.91598064e-02
    1.21747394e-01   1.14677634e-02   6.20704737e-02   6.27509389e-02


In [28]:
# Feature selection using sklearn SelectKBest
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
feature_reg = SelectKBest(f_regression, k=6)
X_new = feature_reg.fit_transform(scaled_features, overall_rating)
X_new

array([[ 0.55172414,  0.61702128,  0.64893617,  0.47826087,  0.37974684,
         0.55208333],
       [ 0.55172414,  0.61702128,  0.64893617,  0.47826087,  0.37974684,
         0.55208333],
       [ 0.46551724,  0.61702128,  0.64893617,  0.47826087,  0.37974684,
         0.55208333],
       ..., 
       [ 0.70689655,  0.86170213,  0.74468085,  0.88043478,  0.72151899,
         0.90625   ],
       [ 0.72413793,  0.88297872,  0.74468085,  0.93478261,  0.65822785,
         0.90625   ],
       [ 0.72413793,  0.90425532,  0.79787234,  0.93478261,  0.62025316,
         0.90625   ]])

In [30]:
# Train and predict model on Decision tree
from sklearn import tree
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.cross_validation import train_test_split
from time import time

reg1 = tree.DecisionTreeClassifier()
reg2 = linear_model.SGDRegressor()

regs = {reg1:"Decision Tree", reg2:"SGDRegressor"}

for key in regs:
    t0 = time()
    X_train, X_test, y_train, y_test = train_test_split(scaled_features, overall_rating, test_size=0.25, random_state=0)

    print "--------------------"
    print regs[key]
    print "--------------------"
    print "Time taken to split the data: {}".format(time()-t0)

    t1 = time()
    key.fit(X_train, y_train)
    print "Time taken to train the model: {}".format(time()-t1)

    t2 = time()
    pred = key.predict(X_test)
    print "Time taken to predict the model: {}".format(time()-t2)

    t3 = time()
    print "r2 score of this model is: {}".format(r2_score(y_test, pred))
    print "Time taken to find the accuracy of model: {}".format(time()-t3)

--------------------
Decision Tree
--------------------
Time taken to split the data: 0.537999868393
Time taken to train the model: 8.97099995613
Time taken to predict the model: 0.144999980927
r2 score of this model is: 0.949239765897
Time taken to find the accuracy of model: 0.010999917984
--------------------
SGDRegressor
--------------------
Time taken to split the data: 0.608999967575
Time taken to train the model: 0.869000196457
Time taken to predict the model: 0.0279998779297
r2 score of this model is: 0.836800387107
Time taken to find the accuracy of model: 0.00999999046326


In [33]:
# Use GridSearch to tune the model
def fit_model(X, y):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    from sklearn.cross_validation import ShuffleSplit
    cv_sets = ShuffleSplit(X.shape[0], n_iter = 10, test_size = 0.20, random_state = 0)

    # TODO: Create a decision tree regressor object
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.svm import SVC
    from sklearn import linear_model
    
    
    regressor1 = DecisionTreeRegressor()
    regressor2 = SVC()
    regressor3 = linear_model.SGDRegressor()

    # TODO: Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    tree_params = {'max_depth' : [3, 6, 9, 20, 100], 'min_samples_split':[2, 3, 4, 5]}
    svm_params = {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],'C': [1, 10, 100, 1000]},{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}
    sgd_params = {'loss':['squared_loss', 'huber'], 'penalty': ['none', 'l2', 'l1', 'elasticnet'], 'n_iter':[10, 75, 100, 500]}
    
    
    # TODO: Transform 'performance_metric' into a scoring function using 'make_scorer' 
    from sklearn.metrics import make_scorer
    scoring_fnc = make_scorer(performance_metric)

    # TODO: Create the grid search object
    from sklearn.grid_search import GridSearchCV
    
    # Updated cv_sets and scoring parameter
    grid = GridSearchCV(regressor3, sgd_params, scoring = scoring_fnc, cv = cv_sets)

    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

def performance_metric(y_true, y_predict):
    
    # TODO: Calculate the performance score between 'y_true' and 'y_predict'
    from sklearn.metrics import r2_score
    
    score = r2_score(y_true, y_predict)
    # Return the score
    return score

In [ ]:
from time import time
t0 = time()
grid_reg = fit_model(pca_features, overall_rating)
print grid_reg.score
# grid_pred = grid_reg()
print "Time taken to train and predict using GridSearch: {}".format(time() - t0)
print "Best parameters are: {}".format(grid_reg.get_params())